In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import timeit
from osgeo import gdal, osr
from contextlib import redirect_stdout
import io

from grib_to_geotiff import grib_to_geotiff
from grib_to_geotiff2 import grib_to_geotiff2

In [2]:
grib_file = "hrrr.t02z.wrfprsf00.grib2"
bands = [585, 616, 620, 628, 629, 661, 662, 663, 664]

## Run both functions

In [3]:
def suppress_print_output(func):
    def wrapper(*args, **kwargs):
        with io.StringIO() as buf, redirect_stdout(buf):
            func(*args, **kwargs)
    return wrapper

def f1_verbose():
    for band in bands:
        # print(f"Extracting band {band}")
        grib_to_geotiff(grib_file, "AAA",band)
def f2_verbose():
    grib_to_geotiff2(grib_file, "BBB",bands)

# Wrap the original function with the suppress_print_output function
f1 = suppress_print_output(f1_verbose)
f2 = suppress_print_output(f2_verbose)

In [4]:
f1()

In [5]:
f2()

KeyboardInterrupt: 

## Check Equality

In [6]:
# Extract data from tif file
ds = gdal.Open("AAA.585.tif")
band = ds.GetRasterBand(1)
data1 = band.ReadAsArray()

In [7]:
# Extract data from tif file
ds = gdal.Open("BBB.fmda_bands.tif")
band = ds.GetRasterBand(1)
data2 = band.ReadAsArray()

In [8]:
np.all(data1 == data2)

True

In [9]:
ds = gdal.Open("AAA.628.tif")
band = ds.GetRasterBand(1)
data1 = band.ReadAsArray()
ds = gdal.Open("BBB.fmda_bands.tif")
band = ds.GetRasterBand(4)
data2 = band.ReadAsArray()
np.all(data1 == data2)

True

In [10]:
ds = gdal.Open("AAA.664.tif")
band = ds.GetRasterBand(1)
data1 = band.ReadAsArray()
ds = gdal.Open("BBB.fmda_bands.tif")
band = ds.GetRasterBand(len(bands))
data2 = band.ReadAsArray()
np.all(data1 == data2)

True

## Time Methods

### Time Band Extraction

In [11]:
timeit.timeit(
    f1, number = 20
)

83.8478271

In [12]:
timeit.timeit(
    f2, number = 20
)

24.0528841

### Time Reading

In [16]:
bands

[585, 616, 620, 628, 629, 661, 662, 663, 664]

In [17]:
def r1():
    for band in bands:
        with gdal.Open(f"AAA.{band}.tif") as ds:
        # ds = gdal.Open(f"AAA.{band}.tif")
            band = ds.GetRasterBand(1)
            data1 = band.ReadAsArray()
            # print(hash(data1.tobytes())) # can use to compare results
    
# def r2():
#     with gdal.Open(f"BBB.fmda_bands.tif") as ds:
#     for i,band in enumerate(bands):
#         band = ds.GetRasterBand(i+1)
#         data2 = band.ReadAsArray()
#         # print(hash(data2.tobytes()))

In [18]:
r1()

AttributeError: __enter__

In [14]:
timeit.timeit(
    r1, number = 100
)

10.500918100000007

In [15]:
timeit.timeit(
    r2, number = 100
)

10.302738799999986